# データの前処理
* 本ハンズオンでは Personalize の使い方にフォーカスするため、データの前処理を事前に行っている
* データソースからデータをダウンロードして、処理する方法をこのノートブックに記載する

In [1]:
import pandas as pd

## データの準備
### サンプルデータのダウンロード
* ここではMovielensの10万件の評価データを利用する
* このnotebookではインタラクション、ユーザー、アイテムの3種類のデータを利用する
* まずはデータのダウンロードから
* movielens の ml-100k 10万行のデータセットをダウンロードする

In [2]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2021-12-07 08:17:46--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Server file no newer than local file ‘ml-100k.zip’ -- not retrieving.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  infla

## データ前処理方針
* ml-100k の中から 3 種類のデータを作成する。
  * ユーザが映画に対して、どの時系列で、どんな評価を下したのか、というインタラクションデータ
  * 映画を閲覧したユーザがどのような属性（年齢、性別など）なのかをマスタとして持つユーザデータ
  * 各映画がどのジャンルに属するかをマスタとして持つアイテムデータ

### データの加工
#### インタラクションデータの加工
Personalizeのインタラクションスキーマ定義に合わせるため、EVENT_TYPE,EVENT_VALUEカラムを設定  
ここではEVENT_TYPEにRATING、EVENT_VALUEにRATINGの値を設定している  

※Personalizeのレコメンデーションは基本的に特定のイベントの発生可能性が高い順でレコメンデーションのリストを返す形となっており  
　サンプルデータを例とした場合、予想されるRATINGが高い順で結果リストを返すわけではない。  
　例.ユーザーAが映画１を見たか見ていないか？が考慮され、RATINGが高かったどうかはレコメンデーションのリストに反映されない  
　このため、ユーザーが良いRATINGをしそうな映画のみをレコメンデーションのリストに含めたい場合、  
　RATINGが良いレコード（例えば3.5以上）のみを学習データに含める必要がある
  また、ここではEVENT_TYPEを指定しているが、1種類しかイベントの種類が存在しない場合、EVENT_TYPEを使用しないことも可能
 　

In [3]:
!head -n5 ./ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596


In [4]:
# データの読み込み
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 10)
data

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [5]:
# スキーマ定義に合わせたデータ修正
data['EVENT_TYPE'] = 'RATING'
data = data.rename(columns={'RATING': 'EVENT_VALUE'})
data = data.loc[:, ["USER_ID", "ITEM_ID", "EVENT_TYPE", "EVENT_VALUE", "TIMESTAMP"]]
data.head()

,USER_ID,ITEM_ID,EVENT_TYPE,EVENT_VALUE,TIMESTAMP
0,196,242,RATING,3,881250949
1,186,302,RATING,3,891717742
2,22,377,RATING,1,878887116
3,244,51,RATING,2,880606923
4,166,346,RATING,1,886397596


In [6]:
# 3.5以上のRATINGのデータのみに絞る
data = data[data['EVENT_VALUE'] >= 3.5]                

# ここではトレーニング時間を短くするため、利用するデータ量を1万件に絞る
data_sampled = data.sample(n=10000)
data_sampled.to_csv('data/interaction.csv', index=False)

In [7]:
!head interaction.csv

head: cannot open ‘interaction.csv’ for reading: No such file or directory


#### ユーザデータの加工
csv形式で出力するのみ

In [8]:
!head -n5 ./ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213


In [9]:
user_data = pd.read_csv('./ml-100k/u.user', sep='|', names=['USER_ID', 'AGE', 'GENDER', 'JOB', 'ZIP'])
user_data.to_csv('data/user.csv', index=False)
user_data.head()

,USER_ID,AGE,GENDER,JOB,ZIP
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
!head user.csv

head: cannot open ‘user.csv’ for reading: No such file or directory


#### アイテムデータの加工
* ITEM_ID, GENRE の csv 形式にする必要がある
  * GENRE については複数の GENRE を保つ場合があるが（「子供向け」と「コメディ」など）、その場合は GENRE カラム内にパイプ（|）で区切って複数格納する
* u.item ファイルに、アイテムID,アイテム名、属するジャンル の ID が格納されている
* u.genre ファイルに、各ジャンルのIDとジャンル名のマスタデータが格納されている
* 上記 2 ファイルをマージしてアイテムデータを作成する

In [11]:
!head -n5 ./ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0


In [12]:
item_data = pd.read_csv('./ml-100k/u.item', sep='|', encoding='latin-1', header=None)
item_data = item_data.drop(3, axis=1)
item_data.head()

,0,1,2,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
!cat ml-100k/u.genre

unknown|0
Action|1
Adventure|2
Animation|3
Children's|4
Comedy|5
Crime|6
Documentary|7
Drama|8
Fantasy|9
Film-Noir|10
Horror|11
Musical|12
Mystery|13
Romance|14
Sci-Fi|15
Thriller|16
War|17
Western|18



In [14]:
genre_data = pd.read_csv('./ml-100k/u.genre', sep='|', encoding='latin-1', header=None)
genre_data.head()

,0,1
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [15]:
column_names = genre_data[0].tolist()
print(column_names)

['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [16]:
column_names.insert(0, 'ITEM_ID')
column_names.insert(1, 'TITLE')
column_names.insert(2, 'RELEASE')
column_names.insert(3, 'IMDB_URL')
print(column_names)

['ITEM_ID', 'TITLE', 'RELEASE', 'IMDB_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [17]:
item_data.columns = column_names
item_data.head()

,ITEM_ID,TITLE,RELEASE,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
def process_genre(item_df):

    genre_list = []
    if(item_df["unknown"] == 1):
        genre_list.append("unknown")
    if(item_df["Action"] == 1):
        genre_list.append("Action")
    if(item_df["Adventure"] == 1):
        genre_list.append("Adventure")
    if(item_df["Animation"] == 1):
        genre_list.append("Animation")
    if(item_df["Children's"] == 1):
        genre_list.append("Children's")
    if(item_df["Comedy"] == 1):
        genre_list.append("Comedy")
    if(item_df["Crime"] == 1):
        genre_list.append("Crime")
    if(item_df["Documentary"] == 1):
        genre_list.append("Documentary")
    if(item_df["Drama"] == 1):
        genre_list.append("Drama")
    if(item_df["Fantasy"] == 1):
        genre_list.append("Fantasy")
    if(item_df["Film-Noir"] == 1):
        genre_list.append("Film-Noir")
    if(item_df["Horror"] == 1):
        genre_list.append("Horror")
    if(item_df["Musical"] == 1):
        genre_list.append("Musical")
    if(item_df["Mystery"] == 1):
        genre_list.append("Mystery")
    if(item_df["Romance"] == 1):
        genre_list.append("Romance")
    if(item_df["Sci-Fi"] == 1):
        genre_list.append("Sci-Fi")
    if(item_df["Thriller"] == 1):
        genre_list.append("Thriller")
    if(item_df["War"] == 1):
        genre_list.append("War")
    if(item_df["Western"] == 1):
        genre_list.append("Western")
        
    return '|'.join(genre_list)



In [19]:
item_data["GENRE"] = item_data.apply( process_genre, axis=1)

In [20]:
item_data_csv = item_data[['ITEM_ID', 'GENRE']]
item_data_csv.head()

,ITEM_ID,GENRE
0,1,Animation|Children's|Comedy
1,2,Action|Adventure|Thriller
2,3,Thriller
3,4,Action|Comedy|Drama
4,5,Crime|Drama|Thriller


In [21]:
item_data_csv.to_csv('data/item.csv', index=False)

In [22]:
!head data/item.csv

ITEM_ID,GENRE
1,Animation|Children's|Comedy
2,Action|Adventure|Thriller
3,Thriller
4,Action|Comedy|Drama
5,Crime|Drama|Thriller
6,Drama
7,Drama|Sci-Fi
8,Children's|Comedy|Drama
9,Drama
